In [14]:
import matplotlib.pyplot as plt
import torch
from torch.quasirandom import SobolEngine
from matplotlib import pyplot as plt
import numpy as np
from alse.accuracy import get_accuracy
from alse.utils import identify_samples_which_satisfy_constraints, store_param, transform, un_transform
# from mpl_toolkits import mplot3d
# from matplotlib import cm
from alse.print_workflow.read_excel import read_excel
from alse.print_workflow.init_model import initialize_models
from alse.one_iter_eci import one_iter_eci

In [15]:
tkwargs = {
    "device": torch.device("cpu"),
    "dtype": torch.float,
}

Read data

In [16]:
X, width, pow_cap, adhere = read_excel()

Name for the excel sheets are hardcoded in this version
Spot size and feed rate are hardcoded in this version


In [17]:
# Store for un_transform later
xrange_list, xmin_list = store_param(X, 4)

In [18]:
X = transform(X, 4)

# You can verify the Normalization is correct by undoing the above step and comparing to original data
# X = un_transform(X, xrange_list, xmin_list, 4) 

In [19]:
list_of_models = initialize_models(X, width, pow_cap, adhere) # You can put in arbitrary amount of output parameters

In [20]:
punchout_radius = 0.1
bounds = torch.tensor([[0, 0, 0, 0], [1, 1, 1, 1]], **tkwargs) # Because we normalized
lb, ub = bounds


In [21]:
constraints = [("gt", 0.3), ("gt", 20), ("gt", 0.1)]
# normalization
# mean = X.mean(dim=-2, keepdim=True)
# std = X.std(dim=-2, keepdim=True) + 1e-6 # prevent dividing by 0
# X = (X - mean) / std

In [22]:
# How many new points do we want
num_pred = 3
Ytemp = [width, pow_cap, adhere]

Code below will be moved to generate_test_points in the next revision

In [ ]:
for i in range(num_pred):

    X, list_of_models = one_iter_eci(X, constraints, punchout_radius, bounds, Ytemp[0].detach(), Ytemp[1].detach(), Ytemp[2].detach())

    # This part is hardcoded
    Ytemp[0] = torch.cat((Ytemp[0], list_of_models[0](X[-1].unsqueeze(0)).loc.unsqueeze(-1)))
    Ytemp[1] = torch.cat((Ytemp[1], list_of_models[1](X[-1].unsqueeze(0)).loc.unsqueeze(-1)))

    # This is a mess, I'll clean it up later
    _compare = (list_of_models[2](X[-1].unsqueeze(0).float()).loc[0] > list_of_models[2](X[-1].unsqueeze(0).float()).loc[1]).long().unsqueeze(0)
    Ytemp[2] = torch.cat((Ytemp[2], _compare))

In [24]:
X = un_transform(X, xrange_list, xmin_list, 4)

In [25]:
new_N_points = X[-num_pred:]

In [26]:
for i in range(new_N_points.shape[0], 0, -1):
    new_power = round(X[-i][0].item(), 2)
    new_velocity = round(X[-i][1].item(), 2)
    new_spotsize = round(X[-i][2].item(), 2)
    new_feedrate = round(X[-i][3].item(), 2)
    print(f"New X_{new_N_points.shape[0]+1-i}: Power: {new_power}W, Velocity: {new_velocity}mm/min, Spot size: {new_spotsize}mm, Feedrate: {new_feedrate}g/min")

New X_1: Power: 2556.23W, Velocity: 1234.74mm/min, Spot size: 2.75mm, Feedrate: 8.33g/min
New X_2: Power: 1023.88W, Velocity: 878.08mm/min, Spot size: 2.9mm, Feedrate: 9.35g/min
New X_3: Power: 1578.78W, Velocity: 2833.87mm/min, Spot size: 2.79mm, Feedrate: 6.2g/min
